In [67]:
import os
import csv
import time
import requests
import pandas as pd
from datetime import datetime

Personal Information

In [68]:
# API key
api_key = "YOUR_API_KEY"

# File path for the CSV file
file_path = os.path.join(os.path.expanduser("~"), "CryptoData.csv")

Coins of Interest

In [69]:
# List of coins to scrape
coins = ["bitcoin", "ethereum", "solana", "binancecoin"]

Scrape Coin Data

In [70]:
# Function to fetch data for a specific cryptocurrency
def get_coin_data(coin, api_key):
    headers = {
        "x-cg-demo-api-key": api_key,
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
    }
    
    # Retry up to 3 times if request fails
    for _ in range(3):
        response = requests.get(f"https://api.coingecko.com/api/v3/coins/{coin}", headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            
            # Extract required data
            coin_info = {
                "Name": data.get("name", "N/A"),
                "CurrentPrice": data.get("market_data", {}).get("current_price", {}).get("usd", "N/A"),
                "1hPercentageChange": data.get("market_data", {}).get("price_change_percentage_1h_in_currency", {}).get("usd", "N/A"),
                "24hPercentageChange": data.get("market_data", {}).get("price_change_percentage_24h_in_currency", {}).get("usd", "N/A"),
                "7dPercentageChange": data.get("market_data", {}).get("price_change_percentage_7d_in_currency", {}).get("usd", "N/A"),
                "24hVolume": data.get("market_data", {}).get("total_volume", {}).get("usd", "N/A"),
                "MarketCap": data.get("market_data", {}).get("market_cap", {}).get("usd", "N/A"),
                "Date": datetime.now().strftime("%d-%m-%Y %H:%M")
            }
            
            return coin_info
        
        else:
            print(f"Failed to retrieve data for {coin}: HTTP {response.status_code}")
            time.sleep(5)  # Wait 5 seconds before retrying
    
    print(f"Failed to retrieve data for {coin} after multiple attempts")
    return None

def get_crypto_data(api_key, coins):
    """
    Fetches data for a list of cryptocurrencies and prints their retrieval status.

    Parameters:
    api_key (str): The API key for accessing the CoinGecko API.
    coins (list): A list of cryptocurrency identifiers to fetch data for.

    Returns:
    list: A list of dictionaries, each containing data for a specific cryptocurrency.
    """
    coin_data = []

    for coin in coins:
        # Fetch data for each coin and append to the list
        data = get_coin_data(coin, api_key)
        coin_data.append(data)

    return coin_data

Record Data in csv File

In [71]:
def write_csv(coin_data, file_path):
    """
    Writes the cryptocurrency data to a CSV file.

    Parameters:
    coin_data (list): A list of dictionaries containing the cryptocurrency data.
    file_path (str): The path to the CSV file.
    """
    fieldnames = ["Name", "CurrentPrice", "1hPercentageChange", "24hPercentageChange", "7dPercentageChange", "24hVolume", "MarketCap", "Date"]

    # Check if the file exists
    file_exists = os.path.isfile(file_path)

    # Write the data to the CSV file
    with open(file_path, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        # Write the header only if the file does not exist
        if not file_exists:
            writer.writeheader()

        for data in coin_data:
            writer.writerow(data)

Data Fetch and Store Loop

In [72]:
# Ensure the directory exists
directory = os.path.dirname(file_path)
if not os.path.exists(directory):
    os.makedirs(directory)

# Main loop to get data every 262.8 seconds and append to CSV
while True:
    coin_data_list = get_crypto_data(api_key, coins)

    write_csv(coin_data_list, file_path)

    # Wait for 262.8 seconds before fetching the data again to avoid exceeding account usage limits
    time.sleep(262.8)

Failed to retrieve data for solana: HTTP 401
Failed to retrieve data for binancecoin: HTTP 401
